# 12 Makia包作图

## 12.1 样例数据

In [ ]:
using DataFrames, DataFramesMeta, CSV
using CategoricalArrays
dclass = CSV.read("data/class19.csv", DataFrame)
transform!(dclass,
    :sex => (s -> categorical(s)),
    renamecols = false)
transform!(dclass, :sex => (x -> levelcode.(x)) => :sexi,
    :age => categorical => :agec)
println(first(dclass,5))

In [ ]:
using Downloads
urlf = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"
dht = CSV.read(Downloads.download(urlf), DataFrame,
    header=0)
rename!(dht, ["age", "sex", "cp", "trestbps", "chol", 
    "fbs", "restecg", "thalach", "exang", "oldpeak",    
    "slope", "ca", "thal", "num"])
print(first(dht,5))

## 12.2 简单一次性完成的图形

In [ ]:
using DataFrames
using CairoMakie
CairoMakie.activate!()
# 折线图
da0 = copy(dclass)
sort!(da0,:height)
lines(da0[:,:height], da0[:,:weight])

In [ ]:
# 直方图
hist(da0[:,:height], bins = 6)
hist(dht[:,:trestbps], bins=15)

In [ ]:
# 密度估计曲线图
density(dclass[:, :height])

In [ ]:
# 盒形图
boxplot(dht[:, :sex], dht[:, :trestbps])

In [ ]:
# 正态QQ图
# 正态QQ图用来检查输入的变量是否来自正态分布总体， 如果散点与直线比较接近， 则可以认为符合。 
# 如果散点的走向与直线明显偏离， 则认为非正态分布。
qqnorm(dht[:, :trestbps], qqline=:fit)

In [ ]:
# 经验分布函数图
ecdfplot(dht[:, :trestbps])

In [ ]:
# 曲面的等高线图
function surfd()
    n = 100
    xs = range(-pi, pi, n)
    ys = range(-pi, pi, n)
    z = [exp(-0.5*((x + 1)^2 + 0.5*y^2))*cos(4*x) +
        exp(-0.8*(2*x^2 + (y-1)^2))*cos(2*y)
        for x in xs, y in ys]
    return (xs, ys, z)
end
x, y, z = surfd()

contour(x, y, z, levels=20)

In [ ]:
# 曲面的染色等高线图
x, y, z = surfd()
fig, ax, plt = contourf(x, y, z, levels=20)
Colorbar(fig[1,2], plt)
fig

In [ ]:
# 曲面的热力图
x, y, z = surfd()
heatmap(x, y, z)

## 12.3 Makie重要概念和作图步骤
* 绘图板(figure)，相当于绘图用的纸张。
* 坐标系统(axis)，可以定位图形内容，设置大小等。
* 绘图(plots)。具体的散点、连线、曲面、热力图等绘图内容。
* 坐标轴、标题等标注。
* 图例，颜色对应条。
* 小图拼凑。<br>

最基本的步骤是用`Figure()`函数新建绘图板， 用`Axis()`函数在此绘图板上建立一到多个绘图区域并自动生成坐标系统， 用散点、连线等作图函数作图。 也可以省略`Figure()`和`Axis()`调用， 直接用作图函数做出简单图形。

### 画布
用`Figure()`函数新建一个画布， 可以在其中放置坐标轴、散点、连线、标题、图例、颜色代码表等内容。 其中可以用`backgroundcolor`设置背景色， 用`resolution`设置大小， 如(800, 600)，单位是像素

In [ ]:
using CairoMakie
CairoMakie.activate!()

fig1 = Figure(backgroundcolor=:gray, resolution = (800, 300))

### 添加坐标轴
用`Axis()`命令在已建立的画布中建立坐标系。 最常用的输入是选择画布分格的左上角格子， 如fig[1,1]， 没有分格子时fig[1,1]就是画布的全部空间。 这种分格称为“布局”（layout）。<br>
在`Axis()`中用`title`指定标题， `xlabel`指定x轴标签， `ylabel`指定y轴标签

In [ ]:
ax = Axis(fig1[1,1],
    title = "这是标题",
    xlabel = "x",
    ylabel = "y")

### 添加图形内容及颜色设置
散点图可以用`color`指定散点颜色， 用`markersize`指定大小（单位为像素）， 用`marker`指定散点的符号， 用`strokecolor`指定散点轮廓线颜色， 用`strokewidth`指定散点轮廓线宽度

In [ ]:
using CairoMakie
CairoMakie.activate!()
using Colors
da1 = sort!(dclass,:age)
fig = Figure()
ax = Axis(fig[1,1],
    title = "设置颜色的散点图示例",
    xlabel = "x",
    ylabel = "y")
    scatter!(ax, da1[:,:age], da1[:,:height],
    color = :purple, marker = :ucircle, markersize=10)
display(fig)

### 多个图层
只要多次调用`lines!()`, `scatter!()`等函数就可以将多个图形叠加地画在同一坐标系中。<br>
只要在每个图层中用label参数指定一个图层的标签， 然后调用`axislegend()`函数就可以在指定的坐标系统中自动给出图例<br>
`axislegend()`可以用参数`position`设置位置， 用l, c, r表示左右位置， t, c, b表示上下位置

In [ ]:
dline01 = DataFrame(x = 1:5,y = [11, 13, 18, 15, 14])
dline02 = DataFrame(x = [0,1,3,6,7,8], y=[15, 13, 14, 11, 10, 9])
fig = Figure()
ax = Axis(fig[1,1],
    title = "散点图和折线图叠加示例",
    xlabel = "x",
    ylabel = "y")
scatter!(ax, dline01[:,:x], dline01[:,:y], label="Data A")
lines!(ax, dline02[:,:x], dline02[:,:y], color=:red, label="Data B")
axislegend(ax)
display(fig)

### 一次性完成作图

In [ ]:
fig, ax, plt = lines(dline01[:,:x], dline01[:,:y])

### 显示和保存

In [ ]:
save("Picture/testPic.png", fig)

## 12.4 Makie作图定制

In [ ]:
using CairoMakie, DataFrames
CairoMakie.activate!()
using Colors
dline01 = DataFrame(x = 1:5,y = [11, 13, 18, 15, 14])
dline02 = DataFrame(x = [0,1,3,6,7,8], y=[15, 13, 14, 11, 10, 9])

### 在作图函数中设置画布和坐标系统属性
可以用`Figure()`设置与绘图板有关的属性， 包括背景色，像素大小等；<br> 用`Axis()`设置与坐标系统有关的属性， 包括标题、轴标题、刻度设置等。

In [ ]:
fig = Figure(backgroundcolor = :gray80, resolution = (400, 300))
ax = Axis(fig[1,1], title = "测试标题", xlabel = "xx")
plt = scatter!(dline01[:,:x], dline01[:,:y], label="散点图")
display(fig)

返回画布、坐标系统和绘图对象三元组的绘图函数（如`scatter(args...; kwargs)`） 支持一个`figure`选项， 输入为一个命名元组， 其中可以使用与`Figure()`函数类似的关键字参数。<br>
返回画布、坐标系统和绘图对象三元组的绘图函数（如`scatter(args...; kwargs)`）和返回坐标系统和绘图对象二元组的绘图函数（如`scatter(gridposition, args...; kwargs)`） 支持一个`axis`选项，可以在其中指定与坐标系统有关的属性。 属性写成命名元组的格式， 其中的元素值与`Axis()`命令的关键字参数相同。

In [ ]:
scatter(dline01[:,:x], dline01[:,:y], 
    figure = (; backgroundcolor=:gray80, resolution=(400,300)),
    axis = (; title="标题", xticks = 1:5))

### 绘图对象属性
`scatter`等一次性作图函数返回画布、坐标系统、绘图对象的三元组， 在指定布局位置时返回坐标系统、绘图对象的二元组， `scatter!()`这样的可变作图函数也返回绘图对象。 设`plt`为一个绘图对象， 则`plt.attributes`包括了绘图对象的各种属性， 比如颜色，符号，线型，字体等。 

Colors包定义了许多颜色名称， 参见:<br>
https://juliagraphics.github.io/Colors.jl/stable/namedcolors/ <br>
ColorThemes包提供了许多调色盘， 其中的符号如`:viridis`, `:heat`可以用作调色盘参数`colormap`的值， 参见：<br>
https://makie.juliaplots.org/stable/documentation/colors/

### 主题
可以用`set_theme!(...)`函数设置一批公用的的属性，称为主题， 其中关于绘图板的属性直接作为关键字参数输入， 关于坐标系统的选项用命名元组输入到`Axis`参数中。 关于图例的选项用命名元组输入到`Legend`参数中。 最后用没有自变量的`settheme!()`取消这个主题的作用。<br>
可以用`pallette`中的`color`设置主题所用的调色盘， 这在多个图层重复时自动循环使用。

In [ ]:
set_theme!(; backgroundcolor=:gray80, resolution=(400,300), palette = (; color = [:green, :cyan]), 
    Axis = (; xtickgridvisible=true, ytickgridvisible=true,
        xgridstyle=:dot, ygridstyle=:dot),
    Legend = (; bgcolor = (:green, 0.2), framecolor=:yellow))
fig, ax, plt = scatter(dline01[:,:x], dline01[:,:y], label="数据A")
scatter!(dline02[:,:x], dline02[:,:y], label="数据B")
axislegend()
set_theme!()
fig

### 坐标轴设置
* 在`Axis()`函数中可以用`title`设置标题， 用`subtitle`设置子标题（在标题下面一行）， 用`xlabel`设置x轴标签， 用`ylabel`设置y轴标签。<br>
* 可以用`titlealign`指定标题和小标题的对齐方式， 如`:left`, `:right`。 <br>
* 可以用`titlecolor`指定标题颜色， 用`titlefont`指定标题字体， 用`titlesize`指定标题字体大小（像素）。<br>
*  小标题也可以使用用类似选项。 可以用`titlegap`指定标题与小标题之间的间隙（像素）。
* 可以用`xlims!()`设置x轴范围，对x、y可以同时设置limits!(ax, x1, x2, y1, y2)

In [ ]:
fig = Figure()
ax = Axis(fig[1,1],
    title = "简单的折线图示例",
    subtitle = "使用Axis()设置",
    titlesize = 30, subtitlesize = 15,
    titlecolor=:green, subtitlecolor=:pink)
lines!(ax, dline01[:,:x], dline01[:,:y])
xlims!(ax, [0, 10])
display(fig)

在`Axis()`中用`xticks`设置x轴刻度， 用`yticks`设置y轴刻度。<br>
`xticks`的值也可以写成[0, 2, 4, 6, 8, 10]这样的向量值。 也可以指定标签

In [ ]:
fig = Figure()
ax = Axis(fig[1,1], xticks = (0:2:10, ["t0", "t2", "t4", "t6", "t8", "t10"]))
lines!(ax, dline01[:,:x], dline01[:,:y])
xlims!(ax, [0, 10])
display(fig)

因为常用各种等间隔刻度， 所以提供了`MultiplesTicks()`函数用来指定特殊间隔的刻度

In [ ]:
let
    da = DataFrame(x = 0:0.1:(4*pi))
    transform!(da, :x => ByRow(sin) => :y)
    fig = Figure()
    ax = Axis(fig[1,1], xticks = MultiplesTicks(5, pi, "π"))
    lines!(ax, da[:,:x], da[:,:y])
    fig
end

可以在`Axis()`中用`xtickformat`参数指定一个函数， 该函数输入刻度值向量， 返回显示字符串向量。<br>
`xtickformat`还可以指定一个格式字符串，其中可以用如`{:.2f}`这样的方法指定一个刻度值的输出格式

In [ ]:
fig = Figure()
ax = Axis(fig[1,1], xtickformat = (s -> string.(s) .* "E3"))
lines!(ax, dline01[:,:x], dline01[:,:y])
xlims!(ax, 0, 6)
display(fig)

In [ ]:
fig = Figure()
ax = Axis(fig[1,1], xtickformat = "{:.2f}")
lines!(ax, dline01[:,:x], dline01[:,:y])
display(fig)

在`Axis()`中设`yscale = log10`可以对y轴使用对数轴。x轴类似。 可取的值包括`identity`(缺省值), `log10`, `log2`, `log`, `sqrt`, `Makie.logit`

###  内部图例

在具体制作图形的函数（如`lines()`）中用`label`选项为某一层图形指定标签， 然后用`axislegend()`制作某一坐标系统的图例<br>
用`marker`可以修改散点形状， 取值如`:circle`, `:rect`, `:utriangle`, `:dtriangle`, `:diamond`, `:pentagon`, `:cross`, `:xcross`等<br>
用`linestyle`可以修改线型，取值如`nothing`(实线), `:dash`, `:dot`, `:dashdot`, `:dashdotdot`等<br>

In [ ]:
fig = Figure()
ax = Axis(fig[1,1],
    title = "散点图和折线图叠加示例",
    xlabel = "x",
    ylabel = "y")
scatter!(ax, dline01[:,:x], dline01[:,:y],
    marker = :utriangle, label = "数据集A")
lines!(ax, dline02[:,:x], dline02[:,:y], 
    color = :red, linestyle = :dot, label = "数据集B")
axislegend(ax)
display(fig)

### 简单布局

在`Axis()`中输入绘图位置时， 默认使用第[1,1]块， 可以不分割小块。 这里位置可用如[1,2], [2,1], [2, 1:2]， 使用范围时表示对应该范围的小块合并使用。

In [ ]:
fig = Figure()
ax1 = Axis(fig[1, 1:2],
    xlabel="height", ylabel="weight")
ax2 = Axis(fig[1, 3],
    xlabel="age", ylabel="weight")
ax3 = Axis(fig[2, 1:3],
    xlabel="height")
fig

有多个小图时， 如果小图之间的x变量, y变量相同， 可能需要对应的坐标轴使用相同的坐标范围。 用`linkxaxis!(ax1, ax2)`可以使两个小图的x坐标轴联系起来， y轴类似。

In [ ]:
linkxaxes!(ax1, ax2)
fig

### 颜色代码条

热力图之类的图形用不同颜色代表z坐标值大小， 一般应该附上一个将颜色与数值参照对应的颜色条作为说明。 使用函数`Colorbar()`，摆放位置也与指定小图位置类似。

In [ ]:
let
    xs = range(-pi, pi, 100)
    ys = range(-pi, pi, 100)
    z = [exp(-0.1*(x^2 + y^2))*(cos(x) + sin(y)) 
        for x in xs, y in ys]
    fig = Figure()
    ax = Axis(fig[1,1])
    p = contourf!(ax, z, colormap = :heat)
    Colorbar(fig[1,2], p)
    display(fig)
end

### 使用LaTeX公式

In [ ]:
let
    x = 0:0.01:4
    f = x -> exp(-0.2*x) * cos(2*pi*x)
    y = f.(x)
    fig, ax, plt = lines(x, y, 
        label = L"e^{-0.2 x} \cos(2 \pi x)")
    text!(L"f(x) = e^x; \quad  g(x) = \cos x", position = (1.5, 0.9))
    axislegend()
    fig
end